In [32]:
import pandas as pd
import re
import csv
import polars as pl

In [33]:
def payer_category(x):
    payers = {
        'Gross Charge': 'gross',
        'Discounted Cash Price': 'cash',
        'Minimum Negotiated Charge': 'min',
        'Maximum Negotiated Charge': 'max'
    }

    try:
        return payers[x]
    except KeyError:
        return 'payer'

In [34]:
file = 'pricing-transparency-charge.csv'
df = pd.read_csv(file)

In [35]:
cols = df.columns.tolist()
drop_cols = [col for col in cols if 'Unnamed' in col]
df.drop(drop_cols, axis=1, inplace=True)
print(df.columns.tolist())

['CDM Item Number', 'Revenue Code', 'Service ID', 'Service Mod1', 'Service Mod2', 'Service Description', ' Gross Charge ', ' Discounted Cash Price ', ' Minimum Negotiated Charge ', ' Maximum Negotiated Charge ', ' Payer Negotiated Charge: AARP (Plan: All) ', ' Payer Negotiated Charge: AARP- MedicareComplete UnitedHealthCare (Plan: All) ', ' Payer Negotiated Charge: Adminastar Blue Cross of Kentucky (Plan: All) ', ' Payer Negotiated Charge: Aetna (Plan: All) ', ' Payer Negotiated Charge: Aetna Better Health (LA) (Plan: All) ', ' Payer Negotiated Charge: Aetna Medicare Advantage (Plan: All) ', ' Payer Negotiated Charge: Blue Advantage of LA (Plan: All) ', ' Payer Negotiated Charge: Blue Cross Blue Shield of LA (Plan: All) ', ' Payer Negotiated Charge: Blue Cross Blue Shield of LA MCR Adv (Plan: All) ', ' Payer Negotiated Charge: Blue Cross Blue Shield of TX (Plan: All) ', ' Payer Negotiated Charge: CHC Louisiana (Plan: All) ', ' Payer Negotiated Charge: Cigna (Plan: All) ', ' Payer Negot

In [36]:
df.rename(columns={
    'CDM Item Number': 'local_code',
    'Revenue Code': 'rev_code',
    'Service ID': 'code',
    'Service Description': 'description',

}, inplace=True)

In [37]:
cols = df.columns.tolist()
id_vars = cols[:6]
value_vars = cols[6:]
df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='standard_charge')

In [38]:
df['modifiers'] = df['Service Mod1'] + "|" + df['Service Mod2']
df.drop(columns=['Service Mod1', 'Service Mod2'], inplace=True)

In [39]:
df['payer'] = df['payer'].str.strip()

In [40]:
df['hcpcs_cpt'] = df['code'].str[:5]
df.loc[df['code'].str.len() == 7, 'modifiers'] = df['code'].str[-2:]

In [41]:
df['payer_category'] = df['payer'].apply(payer_category)

In [42]:
df['hospital_id'] = '190118'
df['rev_code'] = df['rev_code'].astype(str)
df['rev_code'] = df['rev_code'].str.zfill(4)

df['standard_charge'] = df['standard_charge'].str.replace('$', '').str.replace(',', '')
df['standard_charge'] = df['standard_charge'].str.strip()
df['code'] = df['code'].fillna('')
df['hcpcs_cpt'] = df['hcpcs_cpt'].fillna('')
df['modifiers'].fillna('', inplace=True)


C:\Users\adria\AppData\Local\Temp\ipykernel_12332\3848457064.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['standard_charge'] = df['standard_charge'].str.replace('$', '').str.replace(',', '')


In [43]:
df.loc[df['payer_category'] == 'payer', 'plan'] = 'All'
df['payer'].str.replace(' (Plan: All)', '').str.strip()

C:\Users\adria\AppData\Local\Temp\ipykernel_12332\491345305.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['payer'].str.replace(' (Plan: All)', '').str.strip()


0                                              Gross Charge
1                                              Gross Charge
2                                              Gross Charge
3                                              Gross Charge
4                                              Gross Charge
                                ...                        
199495    Payer Negotiated Charge: WellCare of Louisiana...
199496    Payer Negotiated Charge: WellCare of Louisiana...
199497    Payer Negotiated Charge: WellCare of Louisiana...
199498    Payer Negotiated Charge: WellCare of Louisiana...
199499    Payer Negotiated Charge: WellCare of Louisiana...
Name: payer, Length: 199500, dtype: object

In [44]:
df.dropna(subset=['standard_charge'], inplace=True)
df = df[pd.to_numeric(df['standard_charge'], errors='coerce').notnull()]

In [49]:
df['hcpcs_cpt'].fillna('', inplace=True)
df['code'].fillna('', inplace=True)
df['plan'].fillna('', inplace=True)

df['payer'] = df['payer'].str.replace("Payer Negotiated Charge: ", '')
df['payer'] = df['payer'].str.replace(" (Plan: All)", '')
df['payer'] = df['payer'].str.strip()

C:\Users\adria\AppData\Local\Temp\ipykernel_12332\3693055669.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['payer'] = df['payer'].str.replace(" (Plan: All)", '')


In [50]:
df1 = pl.DataFrame(df)
df1.write_csv('test.csv')
# df.to_csv('desoto.csv', index=False)
